# Scraping Steam User Reviews

###### Using python to scrape user reviews from the Steam store without using the API.

[The Steam store](https://store.steampowered.com/) is the premier online store for video games out there. It's been around since 2003, but in 2013, they added the ability for users to submit their own reviews. These reviews are labeled as either "Recommended" or "Not Recommended" by the reviewer. This gives you a huge amount of data you can use for your next NLP project. Or perhaps you're just curious about one game in particular. Either way, I'm going to so you how to scrap those reviews using python so you can get all the reviews you want easily.

Accessing reviews from one game is pretty easy. Just visit 'https://store.steampowered.com/appreviews/&lt;appid&gt;?json=1' with the app id of whatever game you want at the end. For example: 'https://store.steampowered.com/appreviews/413150?json=1'. To find the app id of a particular game, just check the url on its store page.

-app-id image

So to get some reviews, it's just as simple as this:

```
import requests
response = requests.get(url='https://store.steampowered.com/appreviews/413150?json=1').json()
```

You need the `.json()` call to get the information out of the request, instead of just `<Response [200]>`.

Even that can be improved, though. Here is a more easily usable function to get the reviews. The 'params' parameter allows us to put in parameters beyone `json=1`, which we will get to soon. The 'headers' parameter tells Steam that it is a browser making the request, not - for example - a python program pretending to be a browser. It may not be necessary here, but it might help if you are scraping lots of reviews at once.

```
import requests

def get_reviews(appid, params={'json':1}):
        url = 'https://store.steampowered.com/appreviews/'
        response = requests.get(url=url+appid, params=params, headers={'User-Agent': 'Mozilla/5.0'})
        return response.json()
```

Back to those parameters I mentioned. You can see the documentation [here](https://partner.steamgames.com/doc/store/getreviews), but the most important is 'cursor'. If you want more than the maximum 100 reviews you can get from a single request, you'll need to use the cursor. A response includes a 'cursor' attribute, marking which review your request completed on. Including the same cursor in your next request's parameters starts the reviews at the same spot, meaning you get a completely new set of reviews. The cursor is a seemingly random string of characters, and may include characters that need to be encoded to work with a URL request.

```
params = {'json':1}
response = get_reviews(413150)
cursor = response['cursor']
params['cursor'] = cursor.encode()
response_2 = get_reviews(413150, params)
```

### functions

In [ ]:
import requests

In [7]:
def get_reviews(appid, params=None):
        url_start = 'https://store.steampowered.com/appreviews/'
        response = requests.get(url=url_start+appid, params=params, headers={'User-Agent': 'Mozilla/5.0'})
        return response.json() # return data extracted from the json response

In [2]:
def get_n_reviews(appid, n=100):
    reviews = []
    cursor = '*'
    params = { # https://partner.steamgames.com/doc/store/getreviews
            'json' : 1,
            'filter' : 'all', # sort by: recent, updated, all (helpfullness)
            'language' : 'english', # https://partner.steamgames.com/doc/store/localization
            'day_range' : 9223372036854775807, # shows reveiws from all time
            'review_type' : 'all', # all, positive, negative
            'purchase_type' : 'all', # all, non_steam_purchase, steam
        }
    while n > 0:
        params['cursor'] = cursor.encode() # for pagination
        params['num_per_page'] = min(100, n) # 100 is the max possible reviews in one requests
        n -= 100
        
        response = get_reviews(appid, params)
        cursor = response['cursor']
        reviews += response['reviews']
        
        if len(response['reviews']) < 100: break
    
    return reviews

In [ ]:
from bs4 import BeautifulSoup

In [3]:
def get_n_appids(n=100, filter_by='topsellers'):
    appids = []
    url = f'https://store.steampowered.com/search/?category1=998&filter={filter_by}&page='
    page = 0
    
    while page*25 < n:
        page += 1
        response = requests.get(url=url+str(page), headers={'User-Agent': 'Mozilla/5.0'})
        soup = BeautifulSoup(response.text, 'html.parser')
        for row in soup.find_all(class_='search_result_row'):
            appids.append(row['data-ds-appid'])
    
    return appids[:n]

In [ ]:
import pandas as pd

In [4]:
reviews = []
appids = get_n_appids(750)
for appid in appids:
    reviews += get_n_reviews(appid, 100)
df = pd.DataFrame(reviews)[['review', 'voted_up']]
df

NameError: name 'requests' is not defined